# 1. Data Processing

In [11]:
# import the libraries 
import numpy as np
import pandas as pd 
from time import time

In [12]:
def input(filename):
    with open(filename) as f:
        N, D, a, b = [int(x) for x in f.readline().split()]
        F = [[0 for _ in range(D+1)] for _ in range(N+1)]
        for i in range(N):
            d = [int(x) for x in f.readline().split()[:-1]]
            if d:
                F[i][d[0]-1] = 1
    F = np.array(F)
    return N, D, a, b, F


# filename = 'data.txt'
filename = 'data/data-365/dataN50D365.txt'
N, D, a, b, F = input(filename)

In [13]:
# Check status to apply color for each type of status
def status_color(value):
  if value == "Rest": 
    color = 'Green'
  elif value == "Nigh":
    color = 'Red'
  else:
    color = 'White'
  return 'background-color: %s' % color

# 2. Optimization

In [14]:
def select(N, off_today, off_nextday, a, b):
    '''
    :param off_today: number of employees cannot work today
    :param off_nextday: number of employees cannot work on the next day
    :return: z = minimum value of the night shift of an employee
    :return: add = number of employees need to add to suffice the bound
    '''
    z, add = 0, 0
    upper_today = N - off_today - 4*a  # upper bound of the number of employees working today
    lower_today = N - off_today - 4*b

    if upper_today < a or lower_today > b:
        return -1
    else:
        z = max(lower_today, a)

    upper_nextday = N - off_nextday - z - 4*a
    lower_nextday = N - off_nextday - z - 4*b

    if lower_nextday > b:
        z += (lower_nextday - b) # remove redundant employees
    elif upper_nextday < a:
        add = a - upper_nextday  # add employees to suffice the bound
    else:
        add = 0

    if z > b or z < a or add > off_nextday:
        return -1
    else:
        return z, add

In [15]:
def heuristics(N, D, a, b, F):
    num_night = np.full(N, 0)  # number of night shifts of each employee
    global x

    for j in range(D):
        off_today = np.array(F[:, j][:N])
        off_nextday = np.array(F[:, j+1][:N])

        if j != 0:
            for i in range(N):
                if x[i, j-1, 3] == 1:  # if employee i worked at the night shift on the previous day, then rest today
                    off_today[i] = 1

        # Select the possible minimum number of night shift
        if select(N, sum(off_today), sum(off_nextday), a, b) is False:
            print('No optimal solution found.')
            return -1
        else:
            z, add = select(N, sum(off_today), sum(off_nextday), a, b)
        remain = z - add

        # Assign the employee with minimum number of night shift (and absent on the next day) to today's night shift
        emp_off_nextday = np.array([i for i in range(len(off_nextday)) if off_nextday[i] == 1])
        off_night_nextday = np.array([num_night[i] for i in emp_off_nextday])

        while add > 0:
            emp_index = np.argmin(off_night_nextday)
            x[emp_off_nextday[emp_index], j, 3] = 1
            num_night[emp_off_nextday[emp_index]] += 1  # add 1 employee to today's night shift
            off_today[emp_off_nextday[emp_index]] = 1  # avoid working more than one shift in a day
            add -= 1

        # Assign other employees to the night shift if needed (choose among idle employees for today)
        emp_work_today = np.array([i for i in range(len(off_today)) if off_today[i] != 1])
        work_night_today = np.array([num_night[i] for i in emp_work_today])

        while remain > 0:
            emp_index = np.argmin(work_night_today)
            x[emp_work_today[emp_index], j, 3] = 1
            num_night[emp_work_today[emp_index]] += 1
            off_today[emp_work_today[emp_index]] = 1
            remain -= 1

        # Assign other employees to other shifts of today
        i, k = 0, 0
        while i < N and k < 3:
            if off_today[i] == 0:
                x[i, j, k] = 1
                off_today[i] = 1  # avoid assigning the same employee in a day
                k = (k+1) % 3
            i += 1
    return max(num_night)

In [16]:
if __name__ == '__main__':
    x = np.full((N, D, 4), 0)  # solution matrix

    start = time()
    res = heuristics(N, D, a, b, F)
    end = time()
    print('The optimal value is:', res)
    print('The optimal solution is:')

    for i in range(N):
        for j in range(D):
            for k in range(4):
                if x[i, j, k] == 1:
                    print(f'Staff {i+1}: works on day {j+1}, at shift {k+1}')

    # print(x)
    print('Total execution time:', end-start)

The optimal value is: 8
The optimal solution is:
Staff 1: works on day 1, at shift 4
Staff 1: works on day 3, at shift 1
Staff 1: works on day 4, at shift 1
Staff 1: works on day 5, at shift 1
Staff 1: works on day 6, at shift 1
Staff 1: works on day 7, at shift 1
Staff 1: works on day 8, at shift 1
Staff 1: works on day 9, at shift 1
Staff 1: works on day 10, at shift 1
Staff 1: works on day 11, at shift 1
Staff 1: works on day 12, at shift 1
Staff 1: works on day 13, at shift 1
Staff 1: works on day 14, at shift 1
Staff 1: works on day 15, at shift 1
Staff 1: works on day 16, at shift 1
Staff 1: works on day 17, at shift 1
Staff 1: works on day 18, at shift 1
Staff 1: works on day 19, at shift 1
Staff 1: works on day 20, at shift 1
Staff 1: works on day 21, at shift 1
Staff 1: works on day 22, at shift 1
Staff 1: works on day 23, at shift 1
Staff 1: works on day 24, at shift 1
Staff 1: works on day 25, at shift 1
Staff 1: works on day 26, at shift 1
Staff 1: works on day 27, at shift

Staff 6: works on day 71, at shift 3
Staff 6: works on day 72, at shift 3
Staff 6: works on day 73, at shift 3
Staff 6: works on day 74, at shift 3
Staff 6: works on day 75, at shift 3
Staff 6: works on day 76, at shift 3
Staff 6: works on day 77, at shift 3
Staff 6: works on day 78, at shift 3
Staff 6: works on day 79, at shift 2
Staff 6: works on day 80, at shift 3
Staff 6: works on day 81, at shift 3
Staff 6: works on day 82, at shift 3
Staff 6: works on day 83, at shift 3
Staff 6: works on day 84, at shift 2
Staff 6: works on day 85, at shift 3
Staff 6: works on day 86, at shift 3
Staff 6: works on day 87, at shift 3
Staff 6: works on day 88, at shift 3
Staff 6: works on day 89, at shift 3
Staff 6: works on day 90, at shift 3
Staff 6: works on day 91, at shift 3
Staff 6: works on day 92, at shift 3
Staff 6: works on day 93, at shift 3
Staff 6: works on day 94, at shift 3
Staff 6: works on day 95, at shift 3
Staff 6: works on day 96, at shift 3
Staff 6: works on day 97, at shift 3
S

Staff 10: works on day 148, at shift 1
Staff 10: works on day 149, at shift 1
Staff 10: works on day 150, at shift 1
Staff 10: works on day 151, at shift 3
Staff 10: works on day 152, at shift 2
Staff 10: works on day 153, at shift 2
Staff 10: works on day 154, at shift 2
Staff 10: works on day 155, at shift 2
Staff 10: works on day 156, at shift 2
Staff 10: works on day 157, at shift 2
Staff 10: works on day 158, at shift 2
Staff 10: works on day 159, at shift 2
Staff 10: works on day 160, at shift 4
Staff 10: works on day 162, at shift 1
Staff 10: works on day 163, at shift 1
Staff 10: works on day 164, at shift 1
Staff 10: works on day 165, at shift 1
Staff 10: works on day 166, at shift 1
Staff 10: works on day 167, at shift 1
Staff 10: works on day 168, at shift 1
Staff 10: works on day 169, at shift 1
Staff 10: works on day 170, at shift 1
Staff 10: works on day 171, at shift 1
Staff 10: works on day 172, at shift 1
Staff 10: works on day 173, at shift 1
Staff 10: works on day 17

Staff 14: works on day 226, at shift 2
Staff 14: works on day 227, at shift 2
Staff 14: works on day 228, at shift 2
Staff 14: works on day 229, at shift 2
Staff 14: works on day 230, at shift 2
Staff 14: works on day 231, at shift 2
Staff 14: works on day 232, at shift 2
Staff 14: works on day 233, at shift 2
Staff 14: works on day 234, at shift 2
Staff 14: works on day 235, at shift 2
Staff 14: works on day 236, at shift 2
Staff 14: works on day 237, at shift 2
Staff 14: works on day 238, at shift 2
Staff 14: works on day 239, at shift 2
Staff 14: works on day 240, at shift 2
Staff 14: works on day 241, at shift 2
Staff 14: works on day 242, at shift 2
Staff 14: works on day 243, at shift 2
Staff 14: works on day 244, at shift 2
Staff 14: works on day 245, at shift 2
Staff 14: works on day 246, at shift 2
Staff 14: works on day 247, at shift 2
Staff 14: works on day 248, at shift 2
Staff 14: works on day 249, at shift 2
Staff 14: works on day 250, at shift 2
Staff 14: works on day 25

Staff 18: works on day 299, at shift 3
Staff 18: works on day 300, at shift 3
Staff 18: works on day 301, at shift 2
Staff 18: works on day 302, at shift 1
Staff 18: works on day 303, at shift 1
Staff 18: works on day 304, at shift 1
Staff 18: works on day 305, at shift 1
Staff 18: works on day 306, at shift 1
Staff 18: works on day 307, at shift 1
Staff 18: works on day 308, at shift 1
Staff 18: works on day 309, at shift 1
Staff 18: works on day 310, at shift 1
Staff 18: works on day 311, at shift 1
Staff 18: works on day 312, at shift 1
Staff 18: works on day 313, at shift 1
Staff 18: works on day 314, at shift 1
Staff 18: works on day 315, at shift 1
Staff 18: works on day 316, at shift 1
Staff 18: works on day 317, at shift 1
Staff 18: works on day 318, at shift 4
Staff 18: works on day 320, at shift 3
Staff 18: works on day 321, at shift 3
Staff 18: works on day 322, at shift 3
Staff 18: works on day 323, at shift 3
Staff 18: works on day 324, at shift 3
Staff 18: works on day 32

...

Staff 31: works on day 154, at shift 2
Staff 31: works on day 155, at shift 2
Staff 31: works on day 156, at shift 2
Staff 31: works on day 157, at shift 2
Staff 31: works on day 158, at shift 2
Staff 31: works on day 159, at shift 2
Staff 31: works on day 160, at shift 2
Staff 31: works on day 161, at shift 2
Staff 31: works on day 162, at shift 2
Staff 31: works on day 163, at shift 2
Staff 31: works on day 164, at shift 2
Staff 31: works on day 165, at shift 2
Staff 31: works on day 166, at shift 2
Staff 31: works on day 167, at shift 2
Staff 31: works on day 168, at shift 2
Staff 31: works on day 169, at shift 2
Staff 31: works on day 170, at shift 2
Staff 31: works on day 171, at shift 2
Staff 31: works on day 172, at shift 2
Staff 31: works on day 173, at shift 2
Staff 31: works on day 174, at shift 2
Staff 31: works on day 175, at shift 2
Staff 31: works on day 176, at shift 2
Staff 31: works on day 177, at shift 2
Staff 31: works on day 178, at shift 2
Staff 31: works on day 17

Staff 35: works on day 226, at shift 3
Staff 35: works on day 227, at shift 3
Staff 35: works on day 228, at shift 3
Staff 35: works on day 229, at shift 3
Staff 35: works on day 230, at shift 3
Staff 35: works on day 231, at shift 3
Staff 35: works on day 232, at shift 3
Staff 35: works on day 233, at shift 3
Staff 35: works on day 234, at shift 3
Staff 35: works on day 235, at shift 4
Staff 35: works on day 237, at shift 2
Staff 35: works on day 238, at shift 2
Staff 35: works on day 239, at shift 2
Staff 35: works on day 240, at shift 2
Staff 35: works on day 241, at shift 2
Staff 35: works on day 242, at shift 2
Staff 35: works on day 243, at shift 2
Staff 35: works on day 244, at shift 2
Staff 35: works on day 245, at shift 2
Staff 35: works on day 246, at shift 2
Staff 35: works on day 247, at shift 2
Staff 35: works on day 248, at shift 2
Staff 35: works on day 249, at shift 2
Staff 35: works on day 250, at shift 2
Staff 35: works on day 251, at shift 1
Staff 35: works on day 25

Staff 39: works on day 300, at shift 3
Staff 39: works on day 301, at shift 2
Staff 39: works on day 302, at shift 1
Staff 39: works on day 303, at shift 1
Staff 39: works on day 304, at shift 1
Staff 39: works on day 305, at shift 1
Staff 39: works on day 306, at shift 1
Staff 39: works on day 307, at shift 1
Staff 39: works on day 308, at shift 1
Staff 39: works on day 309, at shift 1
Staff 39: works on day 310, at shift 1
Staff 39: works on day 311, at shift 1
Staff 39: works on day 312, at shift 1
Staff 39: works on day 313, at shift 1
Staff 39: works on day 314, at shift 1
Staff 39: works on day 315, at shift 1
Staff 39: works on day 316, at shift 1
Staff 39: works on day 317, at shift 1
Staff 39: works on day 318, at shift 1
Staff 39: works on day 319, at shift 1
Staff 39: works on day 320, at shift 1
Staff 39: works on day 321, at shift 1
Staff 39: works on day 322, at shift 1
Staff 39: works on day 323, at shift 1
Staff 39: works on day 324, at shift 1
Staff 39: works on day 32

Staff 44: works on day 7, at shift 3
Staff 44: works on day 8, at shift 2
Staff 44: works on day 9, at shift 3
Staff 44: works on day 10, at shift 2
Staff 44: works on day 11, at shift 2
Staff 44: works on day 12, at shift 3
Staff 44: works on day 13, at shift 2
Staff 44: works on day 14, at shift 2
Staff 44: works on day 15, at shift 2
Staff 44: works on day 16, at shift 2
Staff 44: works on day 17, at shift 3
Staff 44: works on day 18, at shift 2
Staff 44: works on day 19, at shift 2
Staff 44: works on day 20, at shift 3
Staff 44: works on day 21, at shift 3
Staff 44: works on day 22, at shift 3
Staff 44: works on day 23, at shift 3
Staff 44: works on day 24, at shift 3
Staff 44: works on day 25, at shift 3
Staff 44: works on day 26, at shift 3
Staff 44: works on day 27, at shift 2
Staff 44: works on day 28, at shift 2
Staff 44: works on day 29, at shift 2
Staff 44: works on day 30, at shift 2
Staff 44: works on day 31, at shift 2
Staff 44: works on day 32, at shift 2
Staff 44: works

Staff 48: works on day 81, at shift 1
Staff 48: works on day 82, at shift 1
Staff 48: works on day 83, at shift 1
Staff 48: works on day 84, at shift 3
Staff 48: works on day 85, at shift 1
Staff 48: works on day 86, at shift 1
Staff 48: works on day 87, at shift 1
Staff 48: works on day 88, at shift 1
Staff 48: works on day 89, at shift 1
Staff 48: works on day 90, at shift 1
Staff 48: works on day 91, at shift 1
Staff 48: works on day 92, at shift 1
Staff 48: works on day 93, at shift 1
Staff 48: works on day 94, at shift 1
Staff 48: works on day 95, at shift 3
Staff 48: works on day 96, at shift 1
Staff 48: works on day 97, at shift 3
Staff 48: works on day 98, at shift 4
Staff 48: works on day 100, at shift 3
Staff 48: works on day 101, at shift 2
Staff 48: works on day 102, at shift 1
Staff 48: works on day 103, at shift 1
Staff 48: works on day 104, at shift 1
Staff 48: works on day 105, at shift 1
Staff 48: works on day 106, at shift 1
Staff 48: works on day 107, at shift 1
Staf

# 3. Visualization

In [17]:
# column 5th is useless
S = np.full((N, D, 5), 0)

for staff in range(N):
    for day in range(D):
        for shift in range(4):
            S[staff, day, shift] = int(x[staff, day, shift])

shifts = np.array(["Morning", "Noon", "Afternoon", "Night"])
days = np.array([f"Day {day}" for day in range(1,D+1)])
day_shifts = np.sum(S, axis=0)
day_shifts_solution = pd.DataFrame(data=day_shifts[:, :4].T, index=shifts, columns=days)

# Visualize number staffs for each shift of day
day_shifts_solution.style.background_gradient(cmap='Pastel2')

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10,Day 11,Day 12,Day 13,Day 14,Day 15,Day 16,Day 17,Day 18,Day 19,Day 20,Day 21,Day 22,Day 23,Day 24,Day 25,Day 26,Day 27,Day 28,Day 29,Day 30,Day 31,Day 32,Day 33,Day 34,Day 35,Day 36,Day 37,Day 38,Day 39,Day 40,Day 41,Day 42,Day 43,Day 44,Day 45,Day 46,Day 47,Day 48,Day 49,Day 50,Day 51,Day 52,Day 53,Day 54,Day 55,Day 56,Day 57,Day 58,Day 59,Day 60,Day 61,Day 62,Day 63,Day 64,Day 65,Day 66,Day 67,Day 68,Day 69,Day 70,Day 71,Day 72,Day 73,Day 74,Day 75,Day 76,Day 77,Day 78,Day 79,Day 80,Day 81,Day 82,Day 83,Day 84,Day 85,Day 86,Day 87,Day 88,Day 89,Day 90,Day 91,Day 92,Day 93,Day 94,Day 95,Day 96,Day 97,Day 98,Day 99,Day 100,Day 101,Day 102,Day 103,Day 104,Day 105,Day 106,Day 107,Day 108,Day 109,Day 110,Day 111,Day 112,Day 113,Day 114,Day 115,Day 116,Day 117,Day 118,Day 119,Day 120,Day 121,Day 122,Day 123,Day 124,Day 125,Day 126,Day 127,Day 128,Day 129,Day 130,Day 131,Day 132,Day 133,Day 134,Day 135,Day 136,Day 137,Day 138,Day 139,Day 140,Day 141,Day 142,Day 143,Day 144,Day 145,Day 146,Day 147,Day 148,Day 149,Day 150,Day 151,Day 152,Day 153,Day 154,Day 155,Day 156,Day 157,Day 158,Day 159,Day 160,Day 161,Day 162,Day 163,Day 164,Day 165,Day 166,Day 167,Day 168,Day 169,Day 170,Day 171,Day 172,Day 173,Day 174,Day 175,Day 176,Day 177,Day 178,Day 179,Day 180,Day 181,Day 182,Day 183,Day 184,Day 185,Day 186,Day 187,Day 188,Day 189,Day 190,Day 191,Day 192,Day 193,Day 194,Day 195,Day 196,Day 197,Day 198,Day 199,Day 200,Day 201,Day 202,Day 203,Day 204,Day 205,Day 206,Day 207,Day 208,Day 209,Day 210,Day 211,Day 212,Day 213,Day 214,Day 215,Day 216,Day 217,Day 218,Day 219,Day 220,Day 221,Day 222,Day 223,Day 224,Day 225,Day 226,Day 227,Day 228,Day 229,Day 230,Day 231,Day 232,Day 233,Day 234,Day 235,Day 236,Day 237,Day 238,Day 239,Day 240,Day 241,Day 242,Day 243,Day 244,Day 245,Day 246,Day 247,Day 248,Day 249,Day 250,Day 251,Day 252,Day 253,Day 254,Day 255,Day 256,Day 257,Day 258,Day 259,Day 260,Day 261,Day 262,Day 263,Day 264,Day 265,Day 266,Day 267,Day 268,Day 269,Day 270,Day 271,Day 272,Day 273,Day 274,Day 275,Day 276,Day 277,Day 278,Day 279,Day 280,Day 281,Day 282,Day 283,Day 284,Day 285,Day 286,Day 287,Day 288,Day 289,Day 290,Day 291,Day 292,Day 293,Day 294,Day 295,Day 296,Day 297,Day 298,Day 299,Day 300,Day 301,Day 302,Day 303,Day 304,Day 305,Day 306,Day 307,Day 308,Day 309,Day 310,Day 311,Day 312,Day 313,Day 314,Day 315,Day 316,Day 317,Day 318,Day 319,Day 320,Day 321,Day 322,Day 323,Day 324,Day 325,Day 326,Day 327,Day 328,Day 329,Day 330,Day 331,Day 332,Day 333,Day 334,Day 335,Day 336,Day 337,Day 338,Day 339,Day 340,Day 341,Day 342,Day 343,Day 344,Day 345,Day 346,Day 347,Day 348,Day 349,Day 350,Day 351,Day 352,Day 353,Day 354,Day 355,Day 356,Day 357,Day 358,Day 359,Day 360,Day 361,Day 362,Day 363,Day 364,Day 365
Morning,17,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16
Noon,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,1

In [18]:
col = np.array([f"Staff {staff}" for staff in range(1,N+1)])
row = days
details_heu = np.full((D,N),"Rest")

for r in range(D):
  for c in range(N):
    for shift in range(1,5):
      if S[c,r,shift-1] == 1:
        details_heu[r,c] = shifts[shift-1]
        break

# Visualize details shift for each staff
pf_details_heu = pd.DataFrame(data = details_heu, index = row, columns = col)
pf_details_heu.style.applymap(status_color)